# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Loading-libraries" data-toc-modified-id="Loading-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading libraries</a></div><div class="lev1 toc-item"><a href="#Creating-the-model" data-toc-modified-id="Creating-the-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating the model</a></div><div class="lev1 toc-item"><a href="#Training-400x300" data-toc-modified-id="Training-400x300-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training 400x300</a></div><div class="lev1 toc-item"><a href="#Predictions" data-toc-modified-id="Predictions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predictions</a></div><div class="lev1 toc-item"><a href="#Training-600x450" data-toc-modified-id="Training-600x450-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training 600x450</a></div><div class="lev1 toc-item"><a href="#Predictions" data-toc-modified-id="Predictions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Predictions</a></div>

# Loading libraries

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3

from keras_tqdm import TQDMNotebookCallback

from datetime import datetime
import os

import numpy as np
import pandas as pd
import math

pd.options.display.max_rows = 40

Using TensorFlow backend.


# Creating the model

In [2]:
base_model = InceptionV3(include_top = False,
                        weights = 'imagenet')

86409216/87910968 [============================>.] - ETA: 0s

In [3]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)
model_final = Model(inputs=base_model.input, outputs=x)

In [5]:
model_final.compile(loss = 'binary_crossentropy',
                   optimizer = SGD(lr = 0.0001, momentum = 0.9, decay = 1e-5),
                   metrics = ['accuracy'])

In [6]:
model_final.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [9]:
datagen = ImageDataGenerator(
    rotation_range                = 20,
    width_shift_range             = 0.2,
    height_shift_range            = 0.2,
    horizontal_flip               = True)

validgen = ImageDataGenerator()

# Training 400x300

In [10]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 400
img_height = 300

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 16
batch_size_val   = 32

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)


train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

Found 1727 images belonging to 2 classes.
Found 568 images belonging to 2 classes.


In [ ]:
checkpoint = ModelCheckpoint("weights-iter-4-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 40, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

Epoch 1/40
179s - loss: 0.6043 - acc: 0.6555 - val_loss: 0.5371 - val_acc: 0.6743


Epoch 2/40
135s - loss: 0.4859 - acc: 0.7778 - val_loss: 0.4101 - val_acc: 0.8169


Epoch 3/40
136s - loss: 0.4047 - acc: 0.8460 - val_loss: 0.3282 - val_acc: 0.9032


Epoch 4/40
135s - loss: 0.3402 - acc: 0.8743 - val_loss: 0.2764 - val_acc: 0.9278


Epoch 5/40
136s - loss: 0.2724 - acc: 0.9097 - val_loss: 0.2330 - val_acc: 0.9349


...

Epoch 34/40
133s - loss: 0.0549 - acc: 0.9838 - val_loss: 0.0556 - val_acc: 0.9806


Epoch 35/40
133s - loss: 0.0575 - acc: 0.9832 - val_loss: 0.0555 - val_acc: 0.9806


Epoch 36/40
132s - loss: 0.0505 - acc: 0.9850 - val_loss: 0.0550 - val_acc: 0.9789


Epoch 37/40
133s - loss: 0.0454 - acc: 0.9855 - val_loss: 0.0509 - val_acc: 0.9789


Epoch 40/40
132s - loss: 0.0412 - acc: 0.9861 - val_loss: 0.0505 - val_acc: 0.9789



In [8]:
model_final.load_weights('weights-iter-4-epoch-39.hdf5')

# Predictions

In [11]:
batch_size_test = 64

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_test,
        class_mode  = "binary",
        shuffle     = False)

test_samples       = len(test_gen.filenames)

Found 1531 images belonging to 1 classes.


In [12]:
preds = model_final.predict_generator(test_gen, math.ceil(test_samples / batch_size_test))

In [13]:
preds_filenames = test_gen.filenames
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)
df_result.to_csv("submission_02.csv", encoding="utf8", index=True)
from IPython.display import FileLink
FileLink('submission_02.csv')

/home/ubuntu/extvol/kaggle-invasive-species-monitoring/submission_02.csv

In [ ]:
# Got 0.99246 on LB

# Training 600x450

In [14]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 600
img_height = 450

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 16
batch_size_val   = 32

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)


train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

Found 1727 images belonging to 2 classes.
Found 568 images belonging to 2 classes.


In [15]:
checkpoint = ModelCheckpoint("weights-iter-5-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 40, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

Epoch 1/40
301s - loss: 0.0905 - acc: 0.9704 - val_loss: 0.0670 - val_acc: 0.9824


Epoch 2/40
278s - loss: 0.0780 - acc: 0.9757 - val_loss: 0.0562 - val_acc: 0.9806


Epoch 3/40
278s - loss: 0.0670 - acc: 0.9763 - val_loss: 0.0514 - val_acc: 0.9824


Epoch 4/40
278s - loss: 0.0607 - acc: 0.9803 - val_loss: 0.0483 - val_acc: 0.9824


Epoch 5/40
278s - loss: 0.0658 - acc: 0.9780 - val_loss: 0.0452 - val_acc: 0.9824


...

Epoch 30/40
275s - loss: 0.0217 - acc: 0.9954 - val_loss: 0.0303 - val_acc: 0.9877


Epoch 31/40
275s - loss: 0.0167 - acc: 0.9948 - val_loss: 0.0307 - val_acc: 0.9877


Epoch 32/40
275s - loss: 0.0156 - acc: 0.9971 - val_loss: 0.0314 - val_acc: 0.9877


Epoch 33/40
275s - loss: 0.0259 - acc: 0.9925 - val_loss: 0.0308 - val_acc: 0.9894


Epoch 34/40
275s - loss: 0.0235 - acc: 0.9907 - val_loss: 0.0309 - val_acc: 0.9894
Epoch 00033: early stopping



In [16]:
model_final.load_weights('weights-iter-5-epoch-32.hdf5')

In [17]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples / batch_size_val))

[0.030787440823574722, 0.98943661971830987]

# Predictions

In [18]:
batch_size_test = 32

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_test,
        class_mode  = "binary",
        shuffle     = False)

test_samples       = len(test_gen.filenames)

Found 1531 images belonging to 1 classes.


In [19]:
preds = model_final.predict_generator(test_gen, math.ceil(test_samples / batch_size_test))

In [20]:
preds_filenames = test_gen.filenames
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)
df_result.to_csv("submission_03.csv", encoding="utf8", index=True)
from IPython.display import FileLink
FileLink('submission_03.csv')

/home/ubuntu/extvol/kaggle-invasive-species-monitoring/submission_03.csv

In [ ]:
# Got 0.99454 on LB